# Timeseries Forecasting on Transaction Data

## 1. Installing Dependencies

In [ ]:
# !pip install datasetsforecast
# !pip install entropyhub
# !pip install sktime

In [ ]:
# Basics
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from tqdm import tqdm
import os
import glob

# Some functions for plotting and stuff
import ts_utils as ts_utils

## 2. Data Preparation

In [ ]:
# Size of the data to read
data_size = 'norm'

# Date of the data to read
data_date = '2110' # '2110' = 21st of October

# Read the data (takes around 2 minutes)
dataset = pd.read_csv(f"~/Thesis/data/eod_balances_{data_date}_{data_size}.csv")

dataset

### 2.1 In-sample and Out-sample split

In [ ]:
# Calculate total amount of timeseries
num_timeseries = len(dataset.columns) - 1

# Specify train test split percentage
train_test_split = 0.8

# Split into train and out of sample test data
num_out_of_sample = int(train_test_split * num_timeseries)

# Create in-sample dataframe
in_sample_data = dataset.iloc[:, : num_out_of_sample + 1] # Training and testing

# Create out-sample dataframe
n = num_timeseries-num_out_of_sample
columns_to_keep = dataset.columns[[0]].tolist() + dataset.columns[-n:].tolist()
out_sample_data = dataset[columns_to_keep]

## 3. In-sample analysis

### 3.1 Train/Test splitting and plotting

In [ ]:
# Change the data to the long format
Y_df = in_sample_data.melt(id_vars=['date'], var_name='unique_id', value_name='y')
Y_df = Y_df.rename(columns={'date':'ds'})

# Convert date column to datetime type
Y_df['ds'] = pd.to_datetime(Y_df['ds'])

In [ ]:
# Define the horizon (12 months of 30 days each)
fh = 30
horizon = 12 * fh

# Identify the unique dates in the dataset
unique_dates = Y_df['ds'].unique()

# Convert to a list and then sort the dates
unique_dates = sorted(list(unique_dates))

# Determine the cutoff date (cutoff at 12 months before the last date in the dataset)
cutoff_date = unique_dates[-(horizon + 1)]

# Training data: all data up to the cutoff date
Y_train_df = Y_df[Y_df['ds'] <= cutoff_date]

In [ ]:
# Initialize lists to store the input and test sets
input_dfs = []
test_dfs = []

# Loop to create the 6 input and test sets
for i in range(6):
    # Determine the start date of the test period
    test_start_date = unique_dates[-(horizon - i * 2 * fh)]
    test_end_date = unique_dates[-(horizon - (i * 2 * fh) - fh)]
    
    # Input data: all data up to the start of the current test period
    input_df = Y_df[Y_df['ds'] <= test_start_date]
    input_dfs.append(input_df)
    
    # Test data: the 30-day period following the start of the test period
    test_df = Y_df[(Y_df['ds'] > test_start_date) & (Y_df['ds'] <= test_end_date)]
    test_dfs.append(test_df)

# Define the 6 input periods
Y_input_df_0 = input_dfs[0]
Y_input_df_1 = input_dfs[1]
Y_input_df_2 = input_dfs[2]
Y_input_df_3 = input_dfs[3]
Y_input_df_4 = input_dfs[4]
Y_input_df_5 = input_dfs[5]

# Define the 6 test periods
Y_test_df_0 = test_dfs[0]
Y_test_df_1 = test_dfs[1]
Y_test_df_2 = test_dfs[2]
Y_test_df_3 = test_dfs[3]
Y_test_df_4 = test_dfs[4]
Y_test_df_5 = test_dfs[5]

In [ ]:
# Timeserie to plot
unique_id = '17'

# Plot the train and test dataframes
ts_utils.plot_train_test_split(Y_input_df_0, Y_test_df_0, unique_id)

## 4. Model Callibration

### 4.1 Function to load the dataframes

In [ ]:
# Loading the dataframe containing predictions
def load_prediction_dataframe(model_name, period):
    
    # Retrieve the file path
    path = f'predictions/{model_name}/insample/{period}/model_preds_2110_norm.csv'
    files = glob.glob(path)

    # Check if we have a single file
    if len(files) == 0:
        print(f"Error: No prediction files found for model {model_name} and period {period}")
        return None
    elif len(files) > 1:
        print(f"Warning: Multiple prediction files found for model {model_name} and period {period}, using the first one.")

    # Retrieve the file name
    filename = files[0]
    print(f"Loading prediction dataframe from {filename}")

    # Convert the csv file to a dataframe
    df_pred = pd.read_csv(filename)
    expected_columns = ['unique_id', 'ds', model_name]  # At minimum

    # Check if we have the correct columns
    if not all(col in df_pred.columns for col in expected_columns):
        print(f"Error: Prediction dataframe {filename} is missing expected columns.")
        print("The expected columns are: ", expected_columns)
        return None

    # Rename columns for 'ARIMA' and 'ETS' models
    if model_name in ['ARIMA', 'ETS']:
        col_mapping = {}
        
        for col in df_pred.columns:
            if f'{model_name}-lo-' in col:
                # 'ARIMA-lo-10' becomes 'ARIMA-lo-90'
                num = col.split('-')[-1]
                new_num = str(100 - int(num))
                new_col = f'{model_name}-lo-{new_num}'
                col_mapping[col] = new_col
            # No changes needed for 'hi' columns
        
        if col_mapping:
            df_pred = df_pred.rename(columns=col_mapping)
            print(f"Columns in {model_name} dataframe have been renamed to match expected format.")

    # Return the dataframe with the predictions
    print(f"Prediction dataframe {filename} loaded successfully with {len(df_pred)} rows.")
    return df_pred

### 4.2 Function to calculate final horizon coverage

In [ ]:
# Function to calculate coverage at a horizon
def calculate_coverage_at_horizon(df_merged, model_name, horizon_days=30):
    
    # Ensure 'ds' is datetime
    df_merged['ds'] = pd.to_datetime(df_merged['ds'])
    
    # Group by 'unique_id' to get min 'ds' for each
    min_ds = df_merged.groupby('unique_id')['ds'].min().reset_index().rename(columns={'ds':'min_ds'})
    
    # Merge back to df_merged
    df_merged = df_merged.merge(min_ds, on='unique_id')
    
    # Compute horizon as number of days since min_ds
    df_merged['horizon'] = (df_merged['ds'] - df_merged['min_ds']).dt.days
    
    # Select rows where horizon == (horizon_days - 1)
    target_horizon = horizon_days - 1  # since we start counting from 0
    df_target = df_merged[df_merged['horizon'] == target_horizon].copy()
    total_count = len(df_target)

    # Check if we indeed have the horizon days
    if total_count == 0:
        print(f"Warning: No data for horizon {horizon_days} days ahead.")
        return {}

    # Initialize the confidence intervals
    coverage_dict = {}
    avg_relative_sizes = {}
    median_relative_sizes = {}
    ci_list = ['60%', '70%', '80%', '90%']

    # Loop over the confidence intervals
    for ci in ci_list:
        # Get lower and upper bound column names
        ci_num = ci.strip('%')
        lo_col = f'{model_name}-lo-{ci_num}'
        hi_col = f'{model_name}-hi-{ci_num}'

        # Check if we have those columns
        if lo_col not in df_target.columns or hi_col not in df_target.columns:
            display(df_target)
            print(f"Warning: Columns {lo_col} or {hi_col} not found in dataframe.")
            continue
    
        # Calculate coverage
        df_target[lo_col] = pd.to_numeric(df_target[lo_col], errors='coerce')
        df_target[hi_col] = pd.to_numeric(df_target[hi_col], errors='coerce')
        df_target['y'] = pd.to_numeric(df_target['y'], errors='coerce')
        valid_rows = df_target.dropna(subset=['y', lo_col, hi_col])
        valid_count = len(valid_rows)

        # Check if we have valid rows for confidence interval
        if valid_count == 0:
            print(f"Warning: No valid rows for confidence interval {ci} at horizon {horizon_days}")
            continue

        # Check if it is in between the confidence interval
        in_interval = ((valid_rows['y'] >= valid_rows[lo_col]) & (valid_rows['y'] <= valid_rows[hi_col])).astype(int)

        # Calculate the percentage
        count_in_interval = in_interval.sum()
        coverage = count_in_interval / valid_count * 100
        coverage_dict[ci] = coverage
        
        print(f"For confidence interval {ci} at horizon {horizon_days} days, {count_in_interval}/{valid_count} ({coverage:.2f}%) of ground truth values are within the predicted interval.")

        # Calculate the interval size and relative interval size
        interval_size = valid_rows[hi_col] - valid_rows[lo_col]
        with np.errstate(divide='ignore', invalid='ignore'):
            relative_interval_size = interval_size / valid_rows['y'].abs()
        
        # Exclude cases where 'y' is zero or close to zero
        relative_interval_size = relative_interval_size.replace([np.inf, -np.inf], np.nan)
        relative_interval_size = relative_interval_size.dropna()
    
        # Compute average and median relative interval size
        avg_relative_size = relative_interval_size.mean()
        median_relative_size = relative_interval_size.median()
    
        # Store the results
        avg_relative_sizes[ci] = avg_relative_size
        median_relative_sizes[ci] = median_relative_size
    
        print(f"Average relative interval size for {ci} confidence interval: {avg_relative_size:.4f}")
        print(f"Median relative interval size for {ci} confidence interval: {median_relative_size:.4f}")
    
    return coverage_dict, avg_relative_sizes, median_relative_sizes

### 4.3 Retrieve model callibration numbers

In [ ]:
# List of models and periods
models = ['ARIMA', 'ETS', 'PatchTST', 'DeepAR', 'TimesNet', 'NHITS', 'Chronos-large', 'Chronos-small', 'Chronos-FT']
periods = ['period01', 'period02', 'period03', 'period04', 'period05', 'period06']
horizon_days = 30

# Map periods to existing test dataframes
period_to_test_df = {
    'period01': Y_test_df_0,
    'period02': Y_test_df_1,
    'period03': Y_test_df_2,
    'period04': Y_test_df_3,
    'period05': Y_test_df_4,
    'period06': Y_test_df_5
}

# Initialize results dataframe
index = pd.MultiIndex.from_product([periods, ['60%', '70%', '80%', '90%']], names=['Period', 'Confidence Interval'])
results_df = pd.DataFrame(index=index, columns=models)

# Initialize dataframes to store average and median relative interval sizes
avg_size_df = pd.DataFrame(index=index, columns=models)
median_size_df = pd.DataFrame(index=index, columns=models)

In [ ]:
# Loop over all the periods
for period in periods:
    print(f"\nProcessing period: {period}")
    
    # Get the test dataframe for the current period
    df_test = period_to_test_df.get(period)

    # Check if we have a test dataframe
    if df_test is None:
        print(f"Error: Test dataframe for {period} not found.")
        continue
    else:
        print(f"Test dataframe for {period} loaded successfully with {len(df_test)} rows.")
        
        # Ensure required columns are present
        expected_columns = ['unique_id', 'ds', 'y']
        
        if not all(col in df_test.columns for col in expected_columns):
            print(f"Error: Test dataframe for {period} is missing required columns.")
            continue

    # Loop over all the models
    for model_name in models:
        print(f"\nProcessing model: {model_name} for period: {period}")

        # Load the prediction dataframe
        df_pred = load_prediction_dataframe(model_name, period)

        # Check if the prediction dataframe is present
        if df_pred is None:
            continue

        # Ensure 'unique_id' is string and 'ds' is datetime
        df_pred['unique_id'] = df_pred['unique_id'].astype('string')
        df_pred['ds'] = pd.to_datetime(df_pred['ds'])
        
        # Merge df_test and df_pred on 'unique_id' and 'ds'
        df_merged = pd.merge(df_test, df_pred, on=['unique_id', 'ds'], how='inner')
        print(f"Merged dataframe has {len(df_merged)} rows.")

        # Check if the merged dataframe is not empty
        if len(df_merged) == 0:
            print(f"Error: Merged dataframe is empty for model {model_name} and period {period}.")
            continue
        
        # Calculate coverage at specified horizon
        coverage_dict, avg_relative_sizes, median_relative_sizes = calculate_coverage_at_horizon(df_merged, model_name, horizon_days=horizon_days)
        
        # Fill in coverage results
        for ci, coverage in coverage_dict.items():
            results_df.loc[(period, ci), model_name] = coverage
        
        # Fill in average relative interval sizes
        for ci, avg_size in avg_relative_sizes.items():
            avg_size_df.loc[(period, ci), model_name] = avg_size
        
        # Fill in median relative interval sizes
        for ci, median_size in median_relative_sizes.items():
            median_size_df.loc[(period, ci), model_name] = median_size

print("\nFinal results:")
print(results_df)
# Save results_df to CSV if needed
results_df.to_csv('coverage_results_at_horizon.csv')
print("\nResults saved to coverage_results_at_horizon.csv")

In [ ]:
# Calculate the mean over the periods for each confidence interval and model
mean_df = results_df.groupby(level='Confidence Interval').mean()

# Calculate the standard deviation over the periods for each confidence interval and model
std_df = results_df.groupby(level='Confidence Interval').std()

# Display the mean dataframe
print("Mean coverage over periods:")
display(mean_df)

# Display the standard deviation dataframe
print("\nStandard deviation of coverage over periods:")
display(std_df)

In [ ]:
def Plotting_Coverage(mean_df, std_df, model_categories, colors):
    # Extract the list of confidence intervals
    confidence_intervals = mean_df.index.tolist()  # ['60%', '70%', '80%', '90%']

    # Set up 2x2 grid of subplots
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16, 12))
    axes = axes.flatten()  # Flatten to 1D array for easy iteration

    # Error bar config
    error_config = {'ecolor': 'black', 'elinewidth': 1, 'capsize': 3}

    # Loop over confidence intervals and corresponding axes
    for ax, ci in zip(axes, confidence_intervals):
        # Extract data for the current confidence interval
        mean_data = mean_df.loc[ci]
        std_data = std_df.loc[ci]

        # Sort models by mean coverage (ascending)
        sorted_mean_data = mean_data.sort_values(ascending=True)
        sorted_std_data = std_data[sorted_mean_data.index]

        models = sorted_mean_data.index.tolist()
        mean_values = sorted_mean_data.values
        std_values = sorted_std_data.values

        # Get colors for the models based on their categories
        colors_list = [colors[model_categories[model]] for model in models]

        # Plot horizontal bars with error bars
        bars = ax.barh(models, mean_values, xerr=std_values, color=colors_list,
                       error_kw=error_config)

        # Set labels and invert y-axis to have highest value at top
        ax.set_xlabel('Percentage (%) of Ground Truth within Confidence Interval', fontsize=11)
        ax.invert_yaxis()

        if ax in axes[:2]:
            ax.set_xlabel('')

        # Set title to include the confidence interval
        ax.set_title(f'Callibration at {ci} Confidence Interval', fontsize=14, pad=15)

        # Add dashed line at the true confidence level
        true_confidence = float(ci.strip('%'))
        ax.axvline(x=true_confidence, color='gray', linestyle='--', linewidth=1)

        # Annotate bars with coverage values
        for i, (v, std) in enumerate(zip(mean_values, std_values)):
            ax.text(v + std + 0.5, i, f'{v:.2f}%', 
                    color='black', va='center', fontsize=10)

        # Set x-axis limits to accommodate error bars and annotations
        max_x = max(mean_values + std_values)
        ax.set_xlim(0, max_x + 10)

    # Prevent overlap
    plt.tight_layout(rect=[0, 0, 1, 0.95], h_pad=2.0)

    # Legend for model categories
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in colors.values()]
    labels = colors.keys()
    fig.legend(handles, labels, loc='upper center', ncol=len(labels), fontsize=12)

    plt.show()

In [ ]:
# Define model categories
model_categories = {
    'Naive': 'Baseline Model',
    'ARIMA': 'Baseline Model',
    'ETS': 'Baseline Model',
    'ES_bu': 'Baseline Model',
    'PatchTST': 'Deep Model',
    'NHITS': 'Deep Model',
    'DeepAR': 'Deep Model',
    'TimesNet': 'Deep Model',
    'ESRNN': 'Deep Model',
    'Chronos (small)': 'Pre-trained Model',
    'Chronos (large)': 'Pre-trained Model',
    'Chronos (FT)': 'Pre-trained Model',
    'Chronos': 'Pre-trained Model',
    'TimesFM': 'Pre-trained Model',
    'TimesFM (FT)': 'Pre-trained Model',
    'Chronos-small': 'Pre-trained Model',
    'Chronos-large': 'Pre-trained Model'
}

# Define lighter colors for each category
colors = {
    'Deep Model': '#FFA07A',
    'Pre-trained Model': '#9370DB',
    'Baseline Model': '#ADD8E6' 
}

In [ ]:
Plotting_Coverage(mean_df, std_df, model_categories, colors)

### Confidence Interval Sizes

In [ ]:
# Calculate the mean over the periods for each confidence interval and model
mean_median_size_df = median_size_df.groupby(level='Confidence Interval').mean()

# Calculate the standard deviation over the periods for each confidence interval and model
std_median_size_df = median_size_df.groupby(level='Confidence Interval').std()

# Display the mean dataframe
print("Mean coverage over periods:")
display(mean_median_size_df)

# Display the standard deviation dataframe
print("\nStandard deviation of coverage over periods:")
display(std_median_size_df)

In [ ]:
def plotting_ci_size(mean_df, std_df, model_categories, colors, confidence_interval):
    # Is the confidence interval is valid
    if confidence_interval not in mean_df.index.tolist():
        raise ValueError(f"Confidence interval must be one of {mean_df.index.tolist()}")

    # Create a single figure
    fig, ax = plt.subplots(figsize=(6,4))

    # Error bar configuration
    error_config = {'ecolor': 'black', 'elinewidth': 1, 'capsize': 3}

    # Extract data for the specified confidence interval
    mean_data = mean_df.loc[confidence_interval]
    std_data = std_df.loc[confidence_interval]

    # Sort models by mean coverage (ascending)
    sorted_mean_data = mean_data.sort_values(ascending=True)
    sorted_std_data = std_data[sorted_mean_data.index]

    models = sorted_mean_data.index.tolist()
    mean_values = sorted_mean_data.values
    std_values = sorted_std_data.values

    # Get colors for the models based on their categories
    colors_list = [colors[model_categories[model]] for model in models]

    # Plot horizontal bars with error bars
    bars = ax.barh(models, mean_values, xerr=std_values, color=colors_list,
                   error_kw=error_config)

    # Set labels and invert y-axis to have highest value at top
    ax.set_xlabel('Confidence Interval Width Relative to Ground Truth (%)', fontsize=10)
    ax.invert_yaxis()

    # Set title to include the confidence interval
    ax.set_title(f'Confidence Interval Sizes at {confidence_interval} Confidence Interval', fontsize=13, pad=15)

    # Annotate bars with coverage values
    for i, (v, std) in enumerate(zip(mean_values, std_values)):
        ax.text(v + std + 0.05, i, f'{v:.2f}%', 
                color='black', va='center', fontsize=8)

    # Calculate better x-axis limits
    max_value_with_error = max(mean_values + std_values)

    # Add 2% padding for annotations and readability
    x_max = min(max_value_with_error * 1.2, max_value_with_error + 2)
    ax.set_xlim(0, x_max)

    # Place legend inside the plot in the top right
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in colors.values()]
    labels = colors.keys()
    ax.legend(handles, labels, loc='upper right', fontsize=8)

    # Adjust layout to prevent overlap
    plt.tight_layout()

    plt.show()

In [ ]:
plotting_ci_size(mean_median_size_df, std_median_size_df, model_categories, colors, confidence_interval='90%')